# Clickhouse Compression Comparison
For this test, I tried several compression codecs and precision reduction methods on 4 MERRA-2 nodes with 9 variables and 4 years of data. For spatial-temporal data in particular, there may be better ways to store it and compress it than a tabular database - see zfp compression and zarr files. But I wanted to check out clickhouse.

## Conclusions
**The most important source of size reduction is eliminating false precision**. The next thing to do is to use the Gorilla XOR codec, unless the data is a simple counter, like a timestamp, in which case use DoubleDelta.

This produced a dataset only 30% the size of the uncompressed data. Snappy compressed parquet was 37%, for comparison.

In [1]:
import pandas as pd
from pathlib import Path
import numpy as np
import altair as alt

In [58]:
df = pd.read_csv('./clickhouse_compression_test.tsv', sep='\t')

In [59]:
df[['precision', 'codec_']] = df['table'].str.split('_', expand=True)[[0,1]]

In [60]:
df.head()

,table,name,type,compressed,uncompressed,ratio,codec,precision,codec_
0,fp16_d_lz,GHLAND,Float32,414803,701184,0.591575,"CODEC(Delta(4), LZ4)",fp16,d
1,full32_d_lz,GHLAND,Float32,529300,701184,0.754866,"CODEC(Delta(4), LZ4)",full32,d
2,full32_dg_lz,GHLAND,Float32,561837,701184,0.801269,"CODEC(Delta(4), Gorilla, LZ4)",full32,dg
3,full32_gorilla_lz,GHLAND,Float32,561588,701184,0.800914,"CODEC(Gorilla, LZ4)",full32,gorilla
4,fp16_gorilla_lz,GHLAND,Float32,343651,701184,0.490101,"CODEC(Gorilla, LZ4)",fp16,gorilla


In [70]:
alt.Chart(df).mark_tick(thickness=4, size=25).encode(
    x='precision:N',
    y='compressed:Q',
    #row='codec_:N',
    color='codec:N',
    column='name:N'
).properties(
    width=100,
    height=500,
)

alt.Chart(...)

## Zoom in on indices with log scale

In [73]:
alt.Chart(df).mark_tick(thickness=4, size=25).encode(
    x='precision:N',
    y=alt.Y('compressed:Q', scale=alt.Scale(type='log', base=2)),
    #row='codec_:N',
    color='codec:N',
    column='name:N',
).transform_filter(
    alt.FieldOneOfPredicate(field='name', oneOf=['lat', 'lon', 'time'])
).properties(
    width=100,
    height=500,
)

alt.Chart(...)

## Minimum Compressed size [MB]

In [74]:
min_size = df.groupby(by='name')['compressed'].min().sum() / 2**20
min_size

2.4049949645996094

In [82]:
uncompressed_size = df.groupby(by='name')['uncompressed'].first().sum() / 2**20
uncompressed_size

8.0244140625

In [79]:
min_size / uncompressed_size

0.2997097290677863

## Gorilla only

In [92]:
g = df.loc[df['codec_'] == 'gorilla'].groupby(by=['precision'])[['compressed', 'uncompressed']].sum()
g['ratio'] = g['compressed'] / g['uncompressed']
g

,compressed,uncompressed,ratio
precision,,,
fp16,3367825,8414208,0.400255
full32,5312795,8414208,0.631408
round,2789912,8414208,0.331572


## Gorilla with DoubleDelta timestamps

In [90]:
gdd = df.query('name != "time" & codec_ == "gorilla"').groupby(by='precision')[['compressed', 'uncompressed']].sum() + df.query('name == "time" & codec_ == "dd"').groupby(by='precision')[['compressed', 'uncompressed']].sum()
gdd

,compressed,uncompressed
precision,,
fp16,3163967,8414208
full32,5108937,8414208
round,2586054,8414208


In [88]:
df.query('name == "time" & codec_ == "dd"').groupby(by='precision')[['compressed', 'uncompressed']].sum()

,compressed,uncompressed
precision,,
fp16,962,701184
full32,962,701184
round,962,701184


In [91]:
gdd['ratio'] = gdd['compressed'] / gdd['uncompressed']
gdd

,compressed,uncompressed,ratio
precision,,,
fp16,3163967,8414208,0.376027
full32,5108937,8414208,0.607180
round,2586054,8414208,0.307344


## Snappy Parquet for comparison

In [93]:
root = Path('/mnt/c/data/merra_texas/')
root.exists()

True

In [94]:
parq = dict(fp16=None, round=None, full32=None)

In [101]:
for key in parq.keys():
    fsize = 0
    for file in (root / key).glob('*.parquet'):
        fsize += file.stat().st_size
    parq[key] = fsize

In [105]:
pq = pd.DataFrame(parq, index=[0]).T
pq['ratio'] = pq[0] / g.loc['fp16', 'uncompressed']
pq

,0,ratio
fp16,3873867,0.460396
round,3123527,0.371221
full32,9616634,1.142904
